In [ ]:
import cv2
from vector import distance,pnt2line
import numpy as np
import sys
import neural_network_predict as nnp
from scipy import ndimage
from keras.models import load_model

model = load_model('scModel.h5');

videoNumber = 0

def getLine(frame,color):
    img = frame.copy()
    
    #Zelena je 0, plava je 1
    img[:, :, color] = 0

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, t = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(t, 1, np.pi / 180, 100, minLineLength, maxLineGap)

    x1 = min(lines[:, 0, 0])
    y1 = max(lines[:, 0, 1])
    x2 = max(lines[:, 0, 2])
    y2 = min(lines[:, 0, 3])
    return [(x1, y1), (x2, y2)]

def predict(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray, 203, 255, cv2.THRESH_BINARY)

    im2, contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if(not contours):
        return -1
    c = max(contours, key=cv2.contourArea)

    x, y, w, h = cv2.boundingRect(c)
    rect = thresh1[y:y + h, x:x + w]

    final = cv2.resize(rect, (28, 28), interpolation=cv2.INTER_LINEAR)

    val = model.predict_classes(final.reshape(1, 1, 28, 28).astype('float32'))
    

    return val

def findResult(path):
    numbers = []
    frameNumber =0
    global videoNumber
    result = 0
    
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    blueLine = getLine(frame,1)
    greenLine = getLine(frame,0)
    
    kernel = np.ones((2,2),np.uint8)
    gray = np.array([230, 230, 230])
    white = np.array([255, 255, 255])

    while 1:
        ret, img = cap.read()

        if ret == False:
            break

        mask = cv2.inRange(img, gray, white)
        maskImg = 1.0 * mask

        maskImg = cv2.dilate(maskImg, kernel)  # cv2.erode(img0,kernel)
        maskImg = cv2.dilate(maskImg, kernel)

        labeled, objCount = ndimage.label(maskImg)
        objects = ndimage.find_objects(labeled)

        for i in range(objCount):
            obj = objects[i]
            (xc, yc) = (int((obj[1].stop + obj[1].start) / 2), int((obj[0].stop + obj[0].start) / 2))
            (height, width) = (int(obj[1].stop - obj[1].start), int(obj[0].stop - obj[0].start))

            if (height > 11 or width > 11):
                number = {'center': (xc, yc), 'size': (height, width), 'frameNumber': frameNumber}
        
                inr = []
                for n in numbers:
                    dist = distance(number['center'], n['center'])
                    if (dist < 19):
                        inr.append(n)
                        
                inrCount = len(inr)
                (x, y) = (obj[1].start, obj[0].start)
                pred = predict(img[y: y + width, x: x + height])
                if inrCount == 0:
                    number['blue'] = False
                    number['green'] = False
                    number['history'] = [{'center': (xc, yc), 'size': (height, width), 'frameNumber': frameNumber, 'val': pred}]
                    
                    numbers.append(number)
                elif inrCount == 1:
                    inr[0]['center'] = number['center']
                    inr[0]['frameNumber'] = frameNumber
                    
                    inr[0]['history'].append({'center': (xc, yc), 'size': (height, width), 'frameNumber': frameNumber,'val': pred})


        for n in numbers:
            if (frameNumber - n['frameNumber'] < 3):
                dist, pnt, r = pnt2line(n['center'], blueLine[0], blueLine[1])
                if r > 0:
                    if (dist < 14):
                        if n['blue'] == False:
                            n['blue'] = True
                            history = [(a['val'])[0] for a in n['history']]
                            val = np.argmax(np.bincount(history))
                            result += val
                            print('+' + str(val))
                            
                    
                dist, pnt, r = pnt2line(n['center'], greenLine[1], greenLine[0])
                if r > 0:
                    if (dist < 14):
                        if n['green'] == False:
                            n['green'] = True
                            history = [(a['val'])[0] for a in n['history']]
                            val = np.argmax(np.bincount(history))
                            result -= val
                            print('-' + str(val))
                            
        cv2.putText(img, text = 'Result: ' + str(result), org = (520, 50), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, color = (90,90,255))    
        cv2.imshow('video-' + str(videoNumber),img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        frameNumber += 1
        
        
    videoNumber += 1
    cap.release()
    cv2.destroyAllWindows()

    print(path + ' rezultat: ' + str(result))
    print('----------------------------------------------------------------------------')

    return result

file = open('out.txt','w')
file.write("RA 170/2015 Milan Skrbic")
file.write('\nfile  sum')

video0 = findResult("videos/video-0.avi")
file.write("\nvideo-0.avi\t" + str(video0))

video1 = findResult("videos/video-1.avi")
file.write("\nvideo-1.avi\t" + str(video1))

video2 = findResult("videos/video-2.avi")
file.write("\nvideo-2.avi\t" + str(video2))

video3 = findResult("videos/video-3.avi")
file.write("\nvideo-3.avi\t" + str(video3))

video4 = findResult("videos/video-4.avi")
file.write("\nvideo-4.avi\t" + str(video4))

video5 = findResult("videos/video-5.avi")
file.write("\nvideo-5.avi\t" + str(video5))

video6 = findResult("videos/video-6.avi")
file.write("\nvideo-6.avi\t" + str(video6))

video7 = findResult("videos/video-7.avi")
file.write("\nvideo-7.avi\t" + str(video7))

video8 = findResult("videos/video-8.avi")
file.write("\nvideo-8.avi\t" + str(video8))

video9 = findResult("videos/video-9.avi")
file.write("\nvideo-9.avi\t" + str(video9))


In [2]:
#!/usr/bin/python
import sys

res = []
n = 0
with open('res.txt') as file:	
    data = file.read()
    lines = data.split('\n')
    for id, line in enumerate(lines):
        if(id>0):
            cols = line.split('\t')
            if(cols[0] == ''):
                continue
            cols[1] = cols[1].replace('\r', '')
            res.append(float(cols[1]))
            n += 1

correct = 0
student = []
student_results = []
with open("out.txt") as file:
    data = file.read()
    lines = data.split('\n')
    for id, line in enumerate(lines):
        cols = line.split('\t')
        if(cols[0] == ''):
            continue
        if(id==0):
            student = cols  
        elif(id>1):
            cols[1] = cols[1].replace('\r', '')
            student_results.append(float(cols[1]))

diff = 0
for index, res_col in enumerate(res):
    diff += abs(res_col - student_results[index])
percentage = 100 - abs(diff/sum(res))*100

print (student)
print ('Procenat tacnosti:\t'+str(percentage))
print ('Ukupno:\t'+str(n))


['RA 170/2015 Milan Skrbic']
Procenat tacnosti:	78.9156626506024
Ukupno:	10


In [ ]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import cv2


def preprocess(img):
    ret, thresh1 = cv2.threshold(img, 190, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    c = max(contours, key=cv2.contourArea)

    x, y, w, h = cv2.boundingRect(c)
    rect = thresh1[y:y+h,x:x+w]

    final = cv2.resize(rect, (28, 28), interpolation=cv2.INTER_NEAREST)

    return final

K.set_image_dim_ordering('th')

seed = 7
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
print(X_train.shape)
print(X_train[0].size)
for i in range(len(X_train)):
    X_train[i] = preprocess(X_train[i])

for i in range(len(X_test)):
    X_test[i] = preprocess(X_test[i])

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
X_train = X_train.reshape(X_train.shape[0],1,28,28)
X_test = X_test.reshape(X_test.shape[0],1,28,28)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]


def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

model.save('scModel.h5')
